<a href="https://colab.research.google.com/github/shravanisci/ABC-Immune-Circuit/blob/main/IBD_ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GEOparse

In [ ]:
import GEOparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs("results_ibd", exist_ok=True)


In [ ]:
GSE_ID = "GSE100833"

print("Downloading GEO dataset...")
gse = GEOparse.get_GEO(GSE_ID, destdir=".", annotate_gpl=True)

print("Total samples:", len(gse.gsms))


In [ ]:
expr_dict = {}

for gsm_id, gsm in gse.gsms.items():
    tbl = gsm.table
    if "VALUE" in tbl.columns:
        expr_dict[gsm_id] = tbl["VALUE"].astype(float).values

probe_ids = gse.gsms[list(gse.gsms.keys())[0]].table.iloc[:, 0].values
expr_df = pd.DataFrame(expr_dict, index=probe_ids).T

print("Probe-level matrix:", expr_df.shape)


In [ ]:
gpl = list(gse.gpls.values())[0]
gpl_tbl = gpl.table

probe_to_gene = dict(zip(
    gpl_tbl["ID"],
    gpl_tbl["Gene Symbol"]
))

expr_df.columns = [probe_to_gene.get(p, None) for p in expr_df.columns]
expr_df = expr_df.loc[:, expr_df.columns.notna()]

expr_gene = expr_df.groupby(expr_df.columns, axis=1).mean()
expr_gene.columns = expr_gene.columns.str.upper()

print("Gene-level matrix:", expr_gene.shape)


In [ ]:
genes = ["TNF", "IL6", "IFNG", "IL10"]
present = [g for g in genes if g in expr_gene.columns]

print("Present cytokines:", present)

expr_cyto = expr_gene[present].copy()
print("Cytokine matrix shape:", expr_cyto.shape)


In [ ]:
thresholds_reg = {
    "TNF": expr_cyto["TNF"].quantile(0.75),
    "IL6": expr_cyto["IL6"].quantile(0.75),
    "IFNG": expr_cyto["IFNG"].quantile(0.25)
}

print("Thresholds:", thresholds_reg)


In [ ]:

#regulatory circuit
abc_on_reg = (
    (expr_cyto["TNF"] > thresholds_reg["TNF"]) &
    (expr_cyto["IL6"] > thresholds_reg["IL6"]) &
    (expr_cyto["IFNG"] < thresholds_reg["IFNG"])
).astype(int)

print(f"Regulatory ABC ON: {abc_on_reg.sum()} / {len(abc_on_reg)} ({100*abc_on_reg.mean():.2f}%)")


In [ ]:
thresholds_eff = {
    "TNF": expr_cyto["TNF"].quantile(0.25),
    "IL6": expr_cyto["IL6"].quantile(0.25),
    "IFNG": expr_cyto["IFNG"].quantile(0.75)
}

print("Thresholds:", thresholds_eff)

In [ ]:
#effector circuit
abc_on_eff = (
    (expr_cyto["TNF"] > thresholds_eff["TNF"]) &
    (expr_cyto["IL6"] > thresholds_eff["IL6"]) &
    (expr_cyto["IFNG"] < thresholds_eff["IFNG"])
).astype(int)

print(f"Effector ABC ON: {abc_on_eff.sum()} / {len(abc_on_eff)} ({100*abc_on_eff.mean():.2f}%)")

In [ ]:
import os
os.makedirs("results",
            exist_ok=True)
print("results/ folder ready")

In [ ]:
abc_on_eff.to_csv("results/GSE100833_ABC_effector_state.csv")
abc_on_reg.to_csv("results/GSE100833_ABC_regulatory_state.csv")

In [ ]:
os.listdir("results")

In [ ]:
expr_cyto.assign(ABC_ON=abc_on).to_csv(
    f"results_ibd/{GSE_ID}_abc_results.csv"
)

print("Saved IBD ABC results")


In [ ]:
import os
os.makedirs("results", exist_ok=True)

abc_on_reg.to_csv("results/IBD_regulatory.csv")
abc_on_eff.to_csv("results/IBD_effector.csv")

print(os.listdir("results"))


In [ ]:
import os
print(os.getcwd())


/content
